In [376]:
import pandas as pd
import requests
import json
from simplejson import JSONDecodeError
import warnings
import psycopg2
import yaml

In [381]:
airports_spain = pd.read_csv('./data/airports_spain.csv')[['ICAO', 'City']]

In [378]:
#https://avwx.rest/

with open('./keys/keys.yml') as file:
    keys = yaml.load(file)

headers = {
  'Authorization': keys['key1'],
  'Content-Type': 'application/json'
}

headers2 = {
  'Authorization': keys['key2'],
  'Content-Type': 'application/json'
}

reporting = "true"
formatting = "json"
onfail = "error"

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database="suppliers",
    user=keys['pguser'],
    password=keys['pgpass'])

In [379]:
def parseo (url, city):
    
    #Request
    response = requests.get( url, headers = headers, verify=False )
    if response.status_code == 400 or response.status_code == 204:
        return None
    else:
        json_response = json.loads(response.text)

        #Parsing
        metar = json_response['raw']
        station = json_response['station']
        pressure = json_response['altimeter']['value']
        pressure_unit = json_response['units']['altimeter']
        wind_direction = json_response['wind_direction']['value']
        wind_unit = 'º'
        wind_speed = json_response['wind_speed']['value'] 
        wind_speed_unit = json_response['units']['wind_speed']
        visibility_str = json_response['visibility']['repr']
        visibility_number = json_response['visibility']['value']
        visibility_unit = json_response['units']['visibility']
        temperature = json_response['temperature']['value'] 
        temperature_unit = json_response['units']['temperature']
        remarks = json_response['remarks']
        flight_rules = json_response['flight_rules']
        time = json_response['time']['dt']

        clouds = []
        if not json_response['clouds']:
            clouds = ["No clouds"]
        else:
            for element in json_response['clouds']:
                clouds.append(element['repr'])
        if json_response['wind_gust'] == None:
            json_response['wind_gust'] = 0
        wind_gust = json_response['wind_gust']
        wind_gust_unit = json_response['units']['wind_speed']
        try: 
            wind_direction_var_1 = json_response['wind_variable_direction'][0]['value']
            wind_direction_var_1_unit = 'º'
        except:
            wind_direction_var_1 = None
            wind_direction_var_1_unit = None
        try: 
            wind_direction_var_2 = json_response['wind_variable_direction'][1]['value']
            wind_direction_var_2_unit = 'º'
        except:
            wind_direction_var_2 = None
            wind_direction_var_2_unit = None

        #Storing
        readyforinsertion = {
            'metar': metar,
            'time': time,
            'station': station,
            'city': city,
            'pressure': pressure,
            'pressure_unit': pressure_unit,
            'clouds': clouds,
            'wind_direction': wind_direction,
            'wind_unit': wind_unit,
            'wind_speed': wind_speed,
            'wind_speed_unit': wind_speed_unit,
            'wind_direction_var_1': wind_direction_var_1,
            'wind_direction_var_1_unit': wind_direction_var_1_unit,
            'wind_direction_var_2': wind_direction_var_2,
            'wind_direction_var_2_unit': wind_direction_var_2_unit,
            'visibility_str': visibility_str,
            'visibility_number': visibility_number,
            'visibility_unit': visibility_unit,
            'temperature': temperature,
            'temperature_unit': temperature_unit,
            'remarks': remarks,
            'flight_rules': flight_rules
        }
        return readyforinsertion

In [ ]:
with open('./sql.yml') as file:
    sql_sentences = yaml.load(file)

#cur = conn.cursor()
#cur.execute(sql_sentences['create_table'])
#conn.commit()
#cur.close()

In [380]:
warnings.simplefilter("ignore")

for airport in airports_spain.itertuples():
    url = "https://avwx.rest/api/metar/" + airport[1] + "?reporting=" + reporting + "&format=" + formatting + "&onfail=" + onfail
    data = parseo(url, airport[2])
    if data == None:
        print('Airport ' + airport[1] + ' data not found')
        continue
    else:
        print(data)
        #cur = conn.cursor()
        #cur.execute(sql_sentences['insert_table'], values)
        #conn.commit()
        #cur.close()

{'metar': 'GCFV 091700Z 04018KT 9999 SCT025 26/20 Q1015', 'time': '2020-08-09T17:00:00Z', 'station': 'GCFV', 'city': 'Fuerteventura', 'pressure': 1015, 'pressure_unit': 'hPa', 'clouds': ['SCT025'], 'wind_direction': 40, 'wind_unit': 'º', 'wind_speed': 18, 'wind_speed_unit': 'kt', 'wind_direction_var_1': None, 'wind_direction_var_1_unit': None, 'wind_direction_var_2': None, 'wind_direction_var_2_unit': None, 'visibility_str': '9999', 'visibility_number': 9999, 'visibility_unit': 'm', 'temperature': 26, 'temperature_unit': 'C', 'remarks': '', 'flight_rules': 'VFR'}
{'metar': 'GCHI 091700Z 35019KT 9999 FEW017 SCT022 25/20 Q1017', 'time': '2020-08-09T17:00:00Z', 'station': 'GCHI', 'city': 'Hierro', 'pressure': 1017, 'pressure_unit': 'hPa', 'clouds': ['FEW017', 'SCT022'], 'wind_direction': 350, 'wind_unit': 'º', 'wind_speed': 19, 'wind_speed_unit': 'kt', 'wind_direction_var_1': None, 'wind_direction_var_1_unit': None, 'wind_direction_var_2': None, 'wind_direction_var_2_unit': None, 'visibil

{'metar': 'LEBZ 091700Z 29012KT 250V310 CAVOK 35/12 Q1012', 'time': '2020-08-09T17:00:00Z', 'station': 'LEBZ', 'city': 'Badajoz', 'pressure': 1012, 'pressure_unit': 'hPa', 'clouds': ['No clouds'], 'wind_direction': 290, 'wind_unit': 'º', 'wind_speed': 12, 'wind_speed_unit': 'kt', 'wind_direction_var_1': 250, 'wind_direction_var_1_unit': 'º', 'wind_direction_var_2': 310, 'wind_direction_var_2_unit': 'º', 'visibility_str': 'CAVOK', 'visibility_number': 9999, 'visibility_unit': 'm', 'temperature': 35, 'temperature_unit': 'C', 'remarks': '', 'flight_rules': 'VFR'}
{'metar': 'LECO 091700Z 02007KT 330V070 9999 FEW018 OVC025 21/18 Q1016 NOSIG', 'time': '2020-08-09T17:00:00Z', 'station': 'LECO', 'city': 'La Coruna', 'pressure': 1016, 'pressure_unit': 'hPa', 'clouds': ['FEW018', 'OVC025'], 'wind_direction': 20, 'wind_unit': 'º', 'wind_speed': 7, 'wind_speed_unit': 'kt', 'wind_direction_var_1': 330, 'wind_direction_var_1_unit': 'º', 'wind_direction_var_2': 70, 'wind_direction_var_2_unit': 'º', '

{'metar': 'LERT 091700Z 27007KT 230V320 CAVOK 26/22 Q1012 NOSIG', 'time': '2020-08-09T17:00:00Z', 'station': 'LERT', 'city': 'Rota', 'pressure': 1012, 'pressure_unit': 'hPa', 'clouds': ['No clouds'], 'wind_direction': 270, 'wind_unit': 'º', 'wind_speed': 7, 'wind_speed_unit': 'kt', 'wind_direction_var_1': 230, 'wind_direction_var_1_unit': 'º', 'wind_direction_var_2': 320, 'wind_direction_var_2_unit': 'º', 'visibility_str': 'CAVOK', 'visibility_number': 9999, 'visibility_unit': 'm', 'temperature': 26, 'temperature_unit': 'C', 'remarks': 'NOSIG', 'flight_rules': 'VFR'}
{'metar': 'LESA 091700Z 23012KT 200V260 9999 FEW035 35/05 Q1015', 'time': '2020-08-09T17:00:00Z', 'station': 'LESA', 'city': 'Salamanca', 'pressure': 1015, 'pressure_unit': 'hPa', 'clouds': ['FEW035'], 'wind_direction': 230, 'wind_unit': 'º', 'wind_speed': 12, 'wind_speed_unit': 'kt', 'wind_direction_var_1': 200, 'wind_direction_var_1_unit': 'º', 'wind_direction_var_2': 260, 'wind_direction_var_2_unit': 'º', 'visibility_st

{'metar': 'GCGM 091700Z 23010KT 9999 FEW012 25/20 Q1017 RMK R09/24007KT 180V350 ARP/23010KT 190V270', 'time': '2020-08-09T17:00:00Z', 'station': 'GCGM', 'city': 'La Gomera', 'pressure': 1017, 'pressure_unit': 'hPa', 'clouds': ['FEW012'], 'wind_direction': 230, 'wind_unit': 'º', 'wind_speed': 10, 'wind_speed_unit': 'kt', 'wind_direction_var_1': None, 'wind_direction_var_1_unit': None, 'wind_direction_var_2': None, 'wind_direction_var_2_unit': None, 'visibility_str': '9999', 'visibility_number': 9999, 'visibility_unit': 'm', 'temperature': 25, 'temperature_unit': 'C', 'remarks': 'RMK R09/24007KT 180V350 ARP/23010KT 190V270', 'flight_rules': 'VFR'}
{'metar': 'LELO 091700Z VRB03KT 9999 FEW045CB 28/21 Q1012', 'time': '2020-08-09T17:00:00Z', 'station': 'LELO', 'city': 'Logroño-Agoncillo', 'pressure': 1012, 'pressure_unit': 'hPa', 'clouds': ['FEW045CB'], 'wind_direction': None, 'wind_unit': 'º', 'wind_speed': 3, 'wind_speed_unit': 'kt', 'wind_direction_var_1': None, 'wind_direction_var_1_unit

In [ ]:
conn.close()